In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key
print(weather_api_key)
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


1e30a20132321dad13d821ca294f2164


In [37]:
# Create a series of scatterplots to showcase the following relationships:
# Temperature (F) vs. Latitude
# Humidity (%) vs. Latitude
# Cloudiness (%) vs Latitude
# Wind Speed (mph) vs. Latitude

In [38]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
#len(cities)
cities

['lazurne',
 'chuy',
 'nanortalik',
 'tiksi',
 'rikitea',
 'ushuaia',
 'cape town',
 'busselton',
 'albany',
 'goycay',
 'georgetown',
 'bethel',
 'mys shmidta',
 'new norfolk',
 'saint-pierre',
 'vostok',
 'utiroa',
 'yellowknife',
 'carnarvon',
 'kodiak',
 'saint-philippe',
 'barentsburg',
 'iranshahr',
 'jamestown',
 'jasper',
 'pringsewu',
 'ahipara',
 'clarence town',
 'port blair',
 'east london',
 'namatanai',
 'mataura',
 'mehamn',
 'aruja',
 'xai-xai',
 'qaanaaq',
 'yirol',
 'mizan teferi',
 'saskylakh',
 'moose factory',
 'esperance',
 'punta arenas',
 'longlac',
 'imeni poliny osipenko',
 'cidreira',
 'castro',
 'makakilo city',
 'vaini',
 'dhola',
 'mar del plata',
 'asau',
 'hithadhoo',
 'scottsburgh',
 'tuktoyaktuk',
 'vardo',
 'visby',
 'general pico',
 'bredasdorp',
 'katsuura',
 'illoqqortoormiut',
 'adeje',
 'husavik',
 'tual',
 'lavrentiya',
 'tateyama',
 'coihaique',
 'butaritari',
 'cookeville',
 'corrente',
 'yulara',
 'arraial do cabo',
 'suluq',
 'sentyabrskiy',

In [45]:
# Perform API Calls
## Perform a weather check on each city using a series of successive API calls.
url = "http://api.openweathermap.org/data/2.5/weather?"
city_id = 117  # the city name is working, the city id is not working... the code above pulls city name
city = "London"
# Build query URL
query_url = f'{url}appid={weather_api_key}&q={city}'       #&id={city_id}'
print(query_url)

# get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()

print(f'weather api response: {weather_json}.')

## Include a print log of each city as it'sbeing processed (with the city number and city name).
#cities = ['London','kapaa']
subset_cities = [cities[0],cities[1],cities[2],cities[3],cities[4]]
city_ids = [117,103, 1,2,3]

http://api.openweathermap.org/data/2.5/weather?appid=1e30a20132321dad13d821ca294f2164&q=London
weather api response: {'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 272.74, 'feels_like': 267.42, 'temp_min': 272.04, 'temp_max': 273.15, 'pressure': 1008, 'humidity': 80}, 'visibility': 10000, 'wind': {'speed': 4.12, 'deg': 60}, 'rain': {'1h': 0.13}, 'clouds': {'all': 90}, 'dt': 1612907679, 'sys': {'type': 1, 'id': 1414, 'country': 'GB', 'sunrise': 1612855529, 'sunset': 1612890245}, 'timezone': 0, 'id': 2643743, 'name': 'London', 'cod': 200}.


In [50]:
# Convert Raw Data to DataFrame
# lat, lon, temp, humidity, cloudiness, windspeed
lon = []
lat = []
temp_k = [] # think about when to convert from kelvin to fahrenheit
humidity = []
clouds = []
windspeed = []

for city in subset_cities:
    query_url = f'{url}appid={weather_api_key}&q={city}'       #&id={city_id}'


    # get weather data
    weather_response = requests.get(query_url)
    weather_json = weather_response.json()
    lon.append(weather_json['coord']['lon'])
    lat.append(weather_json['coord']['lat'])
    temp_k.append(weather_json['main']['temp'])
    humidity.append(weather_json['main']['humidity'])
    clouds.append(weather_json['clouds']['all'])
    windspeed.append(weather_json['wind']['speed']) 
    print(weather_json)
    time.sleep(2)
    

{'coord': {'lon': 32.5295, 'lat': 46.084}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 280.4, 'feels_like': 275.53, 'temp_min': 280.4, 'temp_max': 280.4, 'pressure': 1010, 'humidity': 78, 'sea_level': 1010, 'grnd_level': 1009}, 'visibility': 10000, 'wind': {'speed': 4.98, 'deg': 137}, 'clouds': {'all': 100}, 'dt': 1612908717, 'sys': {'country': 'UA', 'sunrise': 1612933232, 'sunset': 1612969671}, 'timezone': 7200, 'id': 703317, 'name': 'Lazurne', 'cod': 200}
{'coord': {'lon': -53.4616, 'lat': -33.6971}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 294.32, 'feels_like': 294.75, 'temp_min': 294.32, 'temp_max': 294.32, 'pressure': 1015, 'humidity': 81, 'sea_level': 1015, 'grnd_level': 1013}, 'visibility': 10000, 'wind': {'speed': 3.24, 'deg': 116}, 'clouds': {'all': 0}, 'dt': 1612908719, 'sys': {'country': 'UY', 'sunrise': 16128

In [51]:
    
wx_dict = {'City':subset_cities,
          'City ID': city_ids,
          'lon':lon,
          'lat':lat,
          'temp_k':temp_k,
          'humidity':humidity,
          'clouds':clouds,
          'windspeed':windspeed}

wx_df = pd.DataFrame(wx_dict)


## Export the city data into a .csv.
wx_df.to_csv("../output_data/cities_weather.csv", index=False, header=True)

## Display the DataFrame
wx_df

,City,City ID,lon,lat,temp_k,humidity,clouds,windspeed
0,lazurne,117,32.5295,46.0840,280.40,78,100,4.98
1,chuy,103,-53.4616,-33.6971,294.32,81,0,3.24
2,nanortalik,1,-45.2371,60.1432,271.34,86,68,8.23
3,tiksi,2,128.8694,71.6872,246.69,82,100,1.45
4,rikitea,3,-134.9692,-23.1203,298.95,71,44,2.72


In [ ]:
# Inspect the data and remove the cities where the humidity > 100%.  UHM, it is not possible to have humidity > 100%?
## Skip this step if there are no cities that have humidity > 100%.

In [ ]:
# Plotting the Data
## Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
## Save the plotted figures as .pngs.